In [25]:
import os
import numpy as np
import trimesh as tm

from pymcfs.mesh import MeshManager, example_mesh
from pymcfs.mcf import mean_curvature_flow
from pymcfs.medial import compute_voronoi_poles
from pymcfs.laplacian import cotangent_laplacian, mean_value_laplacian
from pymcfs.skeleton import thin_mesh, skeletonize, curve_skeleton_from_mesh

import logging
logging.basicConfig(level=logging.INFO)

# mesh = example_mesh("cylinder", radius=1, height=10, sections=32)
mesh = example_mesh("torus", major_radius=1.0, minor_radius=0.3, major_sections=32, minor_sections=8)

In [26]:
mm = MeshManager(mesh)
mm.print_mesh_analysis()
mm.visualize_mesh_3d()


INFO:pymcfs.mesh:Mesh Analysis Report
INFO:pymcfs.mesh:====================
INFO:pymcfs.mesh:
Geometry:
INFO:pymcfs.mesh:  * Vertices: 256
INFO:pymcfs.mesh:  * Faces: 512
INFO:pymcfs.mesh:  * Components: 1
INFO:pymcfs.mesh:  * Volume: 1.59
INFO:pymcfs.mesh:  * Bounds: [-1.3, -1.3, -0.3] to [1.3, 1.3, 0.3]
INFO:pymcfs.mesh:
Mesh Quality:
INFO:pymcfs.mesh:  * Watertight: True
INFO:pymcfs.mesh:  * Winding Consistent: True
INFO:pymcfs.mesh:  * Normal Direction: outward
INFO:pymcfs.mesh:  * Duplicate Vertices: 0
INFO:pymcfs.mesh:  * Degenerate Faces: 0
INFO:pymcfs.mesh:
Topology:
INFO:pymcfs.mesh:  * Genus: 1
INFO:pymcfs.mesh:  * Euler Characteristic: 0
INFO:pymcfs.mesh:
No issues detected
INFO:pymcfs.mesh:
Recommendation:
INFO:pymcfs.mesh:  Mesh appears to be in good condition.
INFO:pymcfs.mesh:====================


In [27]:
V = np.asarray(mesh.vertices)
F = np.asarray(mesh.faces, dtype=int)

# Cotangent Laplacian (robust mode clamps negative cot weights)
L_cot = cotangent_laplacian(V, F, secure=True)
print("Cotangent L: shape=", L_cot.shape, "nnz=", L_cot.nnz)

# Mean-value Laplacian
L_mvl = mean_value_laplacian(V, F)
print("Mean-value L: shape=", L_mvl.shape, "nnz=", L_mvl.nnz)

Cotangent L: shape= (256, 256) nnz= 1792
Mean-value L: shape= (256, 256) nnz= 1792


In [46]:
# Unguided implicit MCF (cotangent, secure weights)
mcf_res = mean_curvature_flow(
    mesh,
    dt=1e-3,
    iterations=300,
    laplacian_type="cotangent",
    laplacian_secure=True,
)

V_after = mcf_res.vertices
mesh_after = tm.Trimesh(vertices=V_after, faces=mesh.faces, process=False)
print("Volume: before=", mesh.volume, " after=", mesh_after.volume)
mm_after = MeshManager(mesh_after)
mm_after.visualize_mesh_3d()

Volume: before= 1.5891804246697518  after= 0.006286035446844755


In [59]:
# "original" guidance pulls vertices back towards the initial shape during flow
mcf_guided = mean_curvature_flow(
    mesh,
    dt=2e-3,
    iterations=5000,
    laplacian_type="cotangent",
    laplacian_secure=True,
    guidance_type="original",
    guidance_weight=0.1,  # increase to preserve more shape
)

V_guid = mcf_guided.vertices
mesh_guid = tm.Trimesh(vertices=V_guid, faces=mesh.faces, process=False)
print("Volume (guided): before=", mesh.volume, " after=", mesh_guid.volume)
mm_guid = MeshManager(mesh_guid)
mm_guid.visualize_mesh_3d()

Volume (guided): before= 1.5891804246697518  after= 1.5544587248686714


In [6]:
# Compute per-vertex medial targets and weights
targets, weights = compute_voronoi_poles(mesh)  # targets: (n,3), weights in [0,1]
print("Voronoi targets/weights:", targets.shape, weights.shape)

# Run MCF with per-vertex diagonal guidance
mcf_medial = mean_curvature_flow(
    mesh,
    dt=1e-2,
    iterations=10,
    laplacian_type="cotangent",
    laplacian_secure=True,
    guidance_targets=targets,
    guidance_diag=weights * 0.5,  # scale weights for overall effect
)
V_medial = mcf_medial.vertices

Voronoi targets/weights: (66, 3) (66,)


In [7]:
V_thin, F_thin = thin_mesh(
    mesh,
    mcf_dt=2e-2,
    mcf_iters=30,
    laplacian_type="cotangent",
    guidance_type=None,       # or "voronoi" with guidance_weight>0 for medial protection
    guidance_weight=0.0,
    collapse_passes=2,
    collapse_percentile=0.3,
    preserve_branch_degree=3,
    collapse_mode="percentile",  # or "pq" / "pq_heap"
    collapse_ratio=0.1,
    medial_protect=False,     # set True with "voronoi" to protect near-pole edges
    medial_protect_threshold=0.5,
    verbose=False,
)
print("Thinned mesh V/F:", V_thin.shape, F_thin.shape)

Thinned mesh V/F: (66, 3) (128, 3)


In [10]:
skel = skeletonize(
    mesh,
    mcf_dt=2e-2,
    mcf_iters=50,
    laplacian_type="cotangent",
    guidance_type=None,        # or "voronoi" with guidance_weight > 0.0
    guidance_weight=0.0,
    build_graph="mesh",        # or "knn"
    knn=12,
    length_quantile=0.7,       # prune leaf stubs
    collapse_passes=3,
    collapse_percentile=0.3,
    preserve_branch_degree=3,
    collapse_mode="percentile",
    collapse_ratio=0.1,
    collapse_domain="graph",   # "graph" or "mesh_only"
    compress_chains=True,
    medial_protect=False,      # pair with guidance_type="voronoi"
    medial_protect_threshold=0.5,
    resample_spacing=None,     # e.g., 0.02 for uniform spacing
    graph_collapse_mode="percentile",
    graph_collapse_ratio=0.1,
    closest_pole_policy=False, # pair with guidance_type="voronoi"
    closest_pole_tol=1.05,
    verbose=True,
)
print("Skeleton: nodes=", skel.nodes.shape, "edges=", skel.edges.shape, "graph_edges=", skel.graph.number_of_edges())
skel.plot_3d();

INFO:pymcfs.skeleton:Skeleton: MCF contraction (dt=0.02, iters=50)
INFO:pymcfs.skeleton:MCF: starting with 66 vertices, 128 faces; dt=0.02, iters=50
INFO:pymcfs.laplacian:Building cotangent Laplacian for 66 vertices, 128 faces
INFO:pymcfs.laplacian:Laplacian built: nnz=450
INFO:pymcfs.laplacian:Mass matrix built: positive entries=66
INFO:pymcfs.skeleton:MCF: factorization complete (A nnz=450)
INFO:pymcfs.skeleton:MCF: completed step 10/50
INFO:pymcfs.skeleton:MCF: completed step 20/50
INFO:pymcfs.skeleton:MCF: completed step 30/50
INFO:pymcfs.skeleton:MCF: completed step 40/50
INFO:pymcfs.skeleton:MCF: completed step 50/50
INFO:pymcfs.skeleton:Skeleton: mesh-edge graph built (nodes=66, edges=192)
INFO:pymcfs.skeleton:Edge-collapse: collapsed 0 edges below thr=0.0645; nodes=66, edges=192
INFO:pymcfs.skeleton:Skeleton: after collapse pass 1, nodes=66, edges=204
INFO:pymcfs.skeleton:Edge-collapse: collapsed 0 edges below thr=0.0645; nodes=66, edges=204
INFO:pymcfs.skeleton:Skeleton: after

Skeleton: nodes= (66, 3) edges= (204, 2) graph_edges= 204


In [11]:
# You can use the thinned surface to extract a curve skeleton from mesh edges
skel2 = curve_skeleton_from_mesh(V_thin, F_thin, compress_chains=True, resample_spacing=0.2)
print("Curve skeleton from thinned mesh:", skel2.nodes.shape, skel2.edges.shape)
skel2.plot_3d();

Curve skeleton from thinned mesh: (3266, 3) (3392, 2)


In [12]:
# Save as SWC for downstream tools (e.g., neuron pipelines)
swc_path = "../data/swc/demo.swc"
skel.write_swc(swc_path, node_type=0, default_radius=1.0, break_cycles="mst", annotate=True)
print("Wrote SWC to:", swc_path)

Wrote SWC to: ../data/swc/demo.swc
